In [18]:
import os
os.chdir("/home/puneetj/Data_DL")
import numpy as np
from glob import glob
from shutil import copyfile
from vgg_bn import Vgg16BN
from keras.callbacks import ModelCheckpoint

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR
%matplotlib inline

In [22]:
data_path = "/home/puneetj/Cats_Dogs" + '/' 
train_path = data_path + 'train/'
valid_path = data_path + 'validation/'
test_path =  data_path + 'test/'
model_path = data_path + 'models/'
submission_path = data_path + 'submissions/'

# data
img_width, img_height = 224, 224
batch_size = 128
nb_train_samples = 20000
nb_valid_samples = 5000
nb_test_samples = 12500
classes = ["cats", "dogs"]
n_classes = len(classes)

# model
nb_epoch = 10
nb_aug = False
lr = 1

In [20]:
vgg = Vgg16BN(size=(img_width, img_height), n_classes=n_classes,ft = False, batch_size=batch_size, lr=lr,weights_file =
            '/home/puneetj/Cats_Dogs/models/0.06-loss_224x224_10epoch_15aug_1lr_vgg16-bnitr3.h5')
model = vgg.model
model.summary()

poped
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
_____________________________________________________________________________________

/home/puneetj/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [17]:
vgg.model()

AttributeError: Vgg16BN instance has no attribute 'model'

In [8]:
vgg.fit(train_path, valid_path,
          nb_trn_samples=nb_train_samples,
          nb_val_samples=nb_valid_samples,          
          nb_epoch=nb_epoch,
          callbacks=[ckpt],
          aug=nb_aug)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Epoch 1/10
20000/20000 [==============================] - 21788s - loss: 0.1249 - acc: 0.9601 - val_loss: 0.0823 - val_acc: 0.9800
Epoch 2/10
20000/20000 [==============================] - 25741s - loss: 0.0670 - acc: 0.9778 - val_loss: 0.0759 - val_acc: 0.9810
Epoch 3/10
20000/20000 [==============================] - 17606s - loss: 0.0539 - acc: 0.9822 - val_loss: 0.0615 - val_acc: 0.9832
Epoch 4/10
20000/20000 [==============================] - 17572s - loss: 0.0412 - acc: 0.9862 - val_loss: 0.0533 - val_acc: 0.9824
Epoch 5/10
20000/20000 [==============================] - 17611s - loss: 0.0338 - acc: 0.9887 - val_loss: 0.0549 - val_acc: 0.9834
Epoch 6/10
20000/20000 [==============================] - 17647s - loss: 0.0306 - acc: 0.9900 - val_loss: 0.0542 - val_acc: 0.9826
Epoch 7/10
20000/20000 [==============================] - 17622s - loss: 0.0229 - acc: 0.9922 - val_loss: 0.0508 - val_acc: 0.985

In [21]:
info_string = "{0}x{1}_{2}epoch_{3}aug_{4}lr_vgg16-bn".format(img_width, img_height, nb_epoch, nb_aug, lr)
ckpt_fn = model_path + '{val_loss:.2f}-loss_' + info_string + 'itr3.h5'

ckpt = ModelCheckpoint(filepath=ckpt_fn,
                      monitor='val_loss',
                      save_best_only=True,
                      save_weights_only=True)

In [25]:
# generate prediction
predictions, filenames = vgg.test('/home/puneetj/Cats_Dogs/test/',12500 )

print("Averaging Predictions Across Augmentations...")

Found 12500 images belonging to 1 classes.
Averaging Predictions Across Augmentations...


In [26]:
predictions

array([[  7.79513121e-11,   1.00000000e+00],
       [  1.00000000e+00,   2.08797261e-17],
       [  9.98776019e-01,   1.22401002e-03],
       ..., 
       [  1.00000000e+00,   2.08310962e-19],
       [  2.08977053e-05,   9.99979079e-01],
       [  1.67317230e-14,   1.00000000e+00]], dtype=float32)

In [46]:
# clip predictions
c = 0.01
preds = np.clip(predictions, c, 1-c)

In [27]:
sub_file = submission_path + info_string + '5.csv'

with open(sub_file, 'w') as f:
    print("Writing Predictions to CSV...")
    f.write('id,label\n')
    for i, image_name in enumerate(filenames):
        pred = ['%.6f' % p for p in predictions[i, :]]
        if i % 2500 == 0:
            print(i, '/', nb_test_samples)
        f.write('%s,%s\n' % (os.path.basename(image_name).replace('.jpg', ''), (pred[1])))
    print("Done.")

Writing Predictions to CSV...
(0, '/', 12500)
(2500, '/', 12500)
(5000, '/', 12500)
(7500, '/', 12500)
(10000, '/', 12500)
Done.


In [53]:
predictions

array([[  7.60817826e-01,   2.39182174e-01],
       [  5.35900235e-01,   4.64099765e-01],
       [  9.99850929e-01,   1.49106534e-04],
       ..., 
       [  9.99231040e-01,   7.69072212e-04],
       [  9.99916553e-01,   8.34641614e-05],
       [  9.99999940e-01,   6.72104647e-08]], dtype=float32)